In [48]:
from circleci.api import Api
import pprint
from collections import defaultdict

token = open(".env").readlines()[0].split("=")[1].strip()
circleci = Api(token)

# get info about your user
#pprint.pprint(circleci.get_user_info())

# get list of all of your projects
# --> build_num --> get_build_info() -> steps -> output_url -> fetch it -> x[0]["message"] is newline delim string
# --> .outcome == "failed" e.g.

results = defaultdict(lambda: defaultdict(int))

# get results 100 at a time

get_how_many = 10000
per_page = 100
pages = int(get_how_many / per_page)

for i in range(pages):
    for build in circleci.get_project_build_summary(
            "pachyderm", "pachyderm-ci-experiment", limit=100, offset=i*per_page,
        ):
        outcome = build["outcome"]
        build_num = build["build_num"]

        if not outcome == "failed" and not outcome == "success":
            continue

        job = build["workflows"]["job_name"]
        #print(f"build {build_num} {outcome} {job}")
        if job.startswith("test-"):
            results[job][outcome] += 1

# Finding flakiest test suites

The following produces a table of test suites ordered by flakiest suite first.

In [56]:
import pandas as pd
df = pd.DataFrame.from_dict(results)

# Transpose
df = df.T

# NaN -> 0
df = df.fillna(0)

df['pass_rate'] = df['success'] / (df['failed'] + df['success'])

# sort by failed
df = df.sort_values(by=["pass_rate"])

df

,failed,success,pass_rate
test-ADMIN,139.0,0.0,0.000000
test-EXAMPLES,136.0,0.0,0.000000
test-MISC,141.0,0.0,0.000000
test-AUTH2,136.0,0.0,0.000000
test-PFS,138.0,2.0,0.014286
test-PPS6,129.0,10.0,0.071942
test-PPS4,121.0,15.0,0.110294
test-PPS5,117.0,22.0,0.158273
test-PPS1,117.0,23.0,0.164286
test-PPS3,115.0,24.0,0.172662


# Finding flakiest individual tests
Now we fetch the logs for each individual test and find the flakiest individual tests.